<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Read-the-dataset" data-toc-modified-id="Read-the-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read the dataset</a></span></li><li><span><a href="#Extract-loc" data-toc-modified-id="Extract-loc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extract loc</a></span></li><li><span><a href="#Clean-common-variables" data-toc-modified-id="Clean-common-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clean common variables</a></span></li><li><span><a href="#Clean-event-variables" data-toc-modified-id="Clean-event-variables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clean event variables</a></span></li></ul></div>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from collections import OrderedDict

# Read the dataset

In [2]:
NAN_IDENTIFIERS = [
    '[-1] keine Angabe','[-2] trifft nicht zu',
    '[-3] nicht valide',
    '[-5] In Fragebogenversion nicht enthalten',
    '[-8] Frage in diesem Jahr nicht Teil des Frageprograms',
]

MONTH_DICT = OrderedDict([('[1] Januar', 'January'),
                          ('[2] Februar', 'February'),
                          ('[3] Maerz', 'March'),
                          ('[4] April', 'April'),
                          ('[5] Mai', 'May'),
                          ('[6] Juni', 'June'),
                          ('[7] Juli', 'July'),
                          ('[8] August', 'August'),
                          ('[9] September', 'September'),
                          ('[10] Oktober', 'October'),
                          ('[11] November', 'November'),
                          ('[12] Dezember', 'December')])

VARIABLE_DICT_PL = {
    # General variables
    'cid': 'ID_ORIGINAL_HH',  # Case id, id of original household
    'syear': 'YEAR',  # survey year
    'hid': 'ID_HH',  # current household id
    'pid': 'ID',  # permanent personal id
    # Characteristics
    'pla0009': 'GENDER',  # Gender
    'plb0022': 'EMPLOYMENT_STATUS',  # Employment status, potentially survey year
    'plb0304': 'REASON_JOB_TERMINATED',  # Why job terminated
    'pld0131': 'MARITAL_STATUS',  # Marital status
#     'pld0135': 'MARRIED_MONTH_SY',  # Month married survey year
#     'pld0136': 'MARRIED_MONTH_PY',  # Month married previous year
    'pld0141': 'DIVORCED_MONTH_SY',  # Month divorced survey year, doc error
    'pld0142': 'DIVORCED_MONTH_PY',  # Month divorced previous year
    'pld0144': 'SEPARATED_MONTH_SY',  # Month separated survey year
    'pld0145': 'SEPARATED_MONTH_PY',  # Month separated previous year
    'pld0147': 'DEATH_PARTNER_MONTH_SY',  # partner died, survey year
    'pld0148': 'DEATH_PARTNER_MONTH_PY',  # partner died, previous year
    'pld0156': 'HH_COMP_CHANGE_MONTH_PY',  # change of household composition, previous year
    'pld0157': 'HH_COMP_CHANGE_MONTH_SY',  # change of household composition, survey year
    'pld0161': 'DEATH_FATHER_MONTH_SY',  # father died survey year
    'pld0162': 'DEATH_FATHER_MONTH_PY',  # father died previous year
    'pld0164': 'DEATH_MOTHER_MONTH_SY',  # mother died survey year
    'pld0165': 'DEATH_MOTHER_MONTH_PY',  # mother died previous year
    'pld0167': 'DEATH_CHILD_MONTH_SY',  # child died survey year
    'pld0168': 'DEATH_CHILD_MONTH_PY',  # child died previous year
    'pld0170': 'DEATH_HH_PERSON_MONTH_SY',  # person in household died, survey year
    'pld0171': 'DEATH_HH_PERSON_MONTH_PY',  # person in household died, survey year
    'ple0010': 'BIRTH_YEAR',  # birth year
    'ple0041': 'LEGALLY_HANDICAPPED_PERC',  # legally handicapped, reduced employment
    # LoC items
    'plh0005': 'LOC_INFLUENCE_SOCIAL_COND',  # influence on social conditions through involvement
    'plh0128': 'LOC_ACHIEVED_DESERVE',  # have not achieved what I deserve
    'plh0245': 'LOC_LIFES_COURSE',  # my lifes course depends on me
    'plh0246': 'LOC_LUCK',  # what you achieve depends on luck
    'plh0247': 'LOC_OTHERS',  # others make crucial decisions in my life
    'plh0248': 'LOC_SUCCESS',  # success takes hard work
    'plh0249': 'LOC_DOUBT',  # doubt my abilities when problems arise
    'plh0250': 'LOC_POSSIBILITIES',  # possibilities are defined by social conditions
    'plh0251': 'LOC_ABILITIES',  # abilities are more important than effort
    'plh0252': 'LOC_LITTLE_CONTROL',  # little control over my life
    'plb0298': 'LAST_JOB_ENDED_MONTH_PY',  # last job ended, previous year
    'plb0299': 'LAST_JOB_ENDED_MONTH_SY',  # last job ended, survey year
}

RETAINED_COLUMNS_PL = list(VARIABLE_DICT_PL.keys())

VARIABLE_DICT_PGEN = {
    'pid': 'ID',
    'syear': 'YEAR',
    'pgmonth': 'INT_MONTH'
}

RETAINED_COLUMNS_PGEN = list(VARIABLE_DICT_PGEN.keys())

EVENT_VARIABLES = ['DEATH_CHILD', 'DEATH_FATHER', 'DEATH_HH_PERSON',
                   'DEATH_MOTHER', 'DEATH_PARTNER', 'DIVORCED',
                   'HH_COMP_CHANGE', 'LAST_JOB_ENDED', 'SEPARATED']

In [3]:
def fill_with_mode(x):
    try:
        return x.value_counts().index[0]
    except IndexError:
        return np.nan
    

def clean_categoricals_from_multiple_nans(df, nan_list):
    """Cleans categoricals by replacing multiple NaN statements with np.nan
    and then removes the missing categories from the categorical index.
    """
    # Replace different NaN statements with np.nan
    df.replace(to_replace=NAN_IDENTIFIERS, value=np.nan, inplace=True)
    # Remove unused categories in categoricals
    categorical_names = list(df.select_dtypes('category').columns)
    for cat in categorical_names:
        df[cat].cat.remove_unused_categories(inplace=True)

    return df


def reorder_month_categoricals(df):
    month_variables = [i for i in df if '_MONTH' in i]

    for variable in month_variables:
        df[variable].cat.set_categories(MONTH_DICT.keys(), ordered=True,
                                        inplace=True)
        df[variable].cat.rename_categories(MONTH_DICT, inplace=True)

    return df

In [4]:
df = pd.read_stata('../data/pl.dta', columns=RETAINED_COLUMNS_PL)
df = df.rename(columns=VARIABLE_DICT_PL)

df.sort_values(['ID', 'YEAR'], axis='rows', inplace=True)

In [5]:
# Load dataset pgen.dta
pgen = pd.read_stata('../data/pgen.dta',
                     columns=RETAINED_COLUMNS_PGEN)
# Rename columns
pgen = pgen.rename(columns=VARIABLE_DICT_PGEN)
# Merge with df
df = df.merge(pgen, on=['ID', 'YEAR'], how='left')

# Post-merging processing
# Clean categoricals
df = clean_categoricals_from_multiple_nans(df, NAN_IDENTIFIERS)
# Relabel data containing months and make them comparable
df = reorder_month_categoricals(df)

In [6]:
# Dropping observations which have NaNs in their loc elicitations in 2005,
# 2010 or 2015
df = df.loc[
    (df[[i for i in df if 'LOC' in i]].isnull().any(axis=1) & 
     df.YEAR.isin([2005, 2010, 2015])) == 0
]

In [7]:
# Shift LEGALLY_HANDICAPPED_PERC to the next year to be able to compute the
# annual change in disability
df['LEGALLY_HANDICAPPED_PERC_SHIFTED'] = df.groupby(
    'ID').LEGALLY_HANDICAPPED_PERC.transform('shift')

In [8]:
# Select only observations which are in one of the two ranges, 2005-2010 or
# 2010-2015, and which are complete, meaning having 6 observations for one
# range or 11 for two.
# First, create variables to indicate complete ranges
df['YEAR_2005_2010'] = df.YEAR.isin([2005, 2006, 2007, 2008, 2009, 2010])
df['YEAR_2005_2010_SUM'] = df.groupby('ID').YEAR_2005_2010.transform(sum)
df['YEAR_2010_2015'] = df.YEAR.isin([2010, 2011, 2012, 2013, 2014, 2015])
df['YEAR_2010_2015_SUM'] = df.groupby('ID').YEAR_2010_2015.transform(sum)
# Select only valid years when range is complete
df = df.loc[
    (df.YEAR.isin([2005, 2006, 2007, 2008, 2009, 2010]) &
     (df.YEAR_2005_2010_SUM == 6)) |
    (df.YEAR.isin([2010, 2011, 2012, 2013, 2014, 2015]) &
     (df.YEAR_2010_2015_SUM == 6))
]
# Test that for each individual, there are only 6 or 11 possible
# observations. There are 6 if individuals are only observed over one
# period, 2005-2010 or 2010-2015, and 11 if they are observed over the
# whole range.
assert df.groupby('ID').YEAR.count().isin([6, 11]).all()

# Drop temporary columns
df.drop(['YEAR_2005_2010', 'YEAR_2005_2010_SUM',
         'YEAR_2010_2015', 'YEAR_2010_2015_SUM'], axis='columns',
        inplace=True)

# Extract loc

In [9]:
# Copy loc dataframe and drop columns in other frame
loc = df.loc[df.YEAR.isin([2005, 2010, 2015]),
             ['ID', 'YEAR'] + [i for i in df if 'LOC' in i]].copy()
df.drop([i for i in df if 'LOC' in i], axis='columns', inplace=True)

# Clean common variables

## ``BIRTH_YEAR``

In [10]:
df.BIRTH_YEAR.replace(to_replace=-5, value=np.nan, inplace=True)
filled_birth_year = df.groupby('ID').BIRTH_YEAR.transform(fill_with_mode)
df.BIRTH_YEAR = filled_birth_year

# Create age variable
df['AGE'] = df.YEAR - df.BIRTH_YEAR
# Drop BIRTH_YEAR
df.drop('BIRTH_YEAR', axis='columns', inplace=True)

## ``EMPLOYMENT_STATUS``

In [11]:
# EMPLOYMENT_STATUS Create states according to Preuss, Hennecke (2017) who
# sort employed, part-time employed and self-employed to employed,
# unemployed and others.
employment_dict = {
    '[1] Voll erwerbstaetig': 'Employed',
    '[2] Teilzeitbeschaeftigung': 'Employed',
    '[3] Ausbildung,Lehre': 'Other',
    '[4] Geringfuegig beschaeftigt': 'Other',
    '[5] Altersteilzeit mit Arbeitszeit Null': 'Other',
    '[6] Freiwilliger Wehrdienst': 'Other',
    '[7] Freiwsoziales/oekol.Jahr, Bundesfreiwilligendienst': 'Other',
    '[8] Werkstatt fuer behinderte Menschen': 'Other',
    '[9] Nicht erwerbstaetig': 'Not Employed'
}
df.EMPLOYMENT_STATUS.replace(employment_dict, inplace=True)
df.EMPLOYMENT_STATUS = df.EMPLOYMENT_STATUS.astype('category')

## ``GENDER``

In [12]:
gender_dict = {'[1] Maennlich': 'Male', '[2] Weiblich': 'Female'}
df.GENDER.cat.rename_categories(gender_dict, inplace=True)

filled_gender = df.groupby('ID').GENDER.transform(fill_with_mode)
df.GENDER = filled_gender.astype('category')

## ``LEGALLY_HANDICAPPED_PERC``

The question here is what to make of the negative values where people's legal handicapped status was reduced?

In [13]:
# Replace negative values with NaNs
legally_handicapped_perc_dict = {
    -1: 0,   # no answer/don't know
    -2: 0,   # does not apply
    -3: 70,  # implausible value, only applies to ID == 2722302 who has 70%
             # in the previous year. Assume it is constant.
}
df.replace({
    'LEGALLY_HANDICAPPED_PERC': legally_handicapped_perc_dict,
    'LEGALLY_HANDICAPPED_PERC_SHIFTED': legally_handicapped_perc_dict},
    inplace=True)
# Calculate change in LEGALLY_HANDICAPPED_PERC
df['LEGALLY_HANDICAPPED_PERC_CHANGE'] = (
    df.LEGALLY_HANDICAPPED_PERC - df.LEGALLY_HANDICAPPED_PERC_SHIFTED)
# Drop unneccessary columns
df.drop(['LEGALLY_HANDICAPPED_PERC', 'LEGALLY_HANDICAPPED_PERC_SHIFTED'],
        axis='columns', inplace=True)

## ``MARITAL_STATUS``

In [14]:
# MARITAL STATUS
marital_status_dict = {
    '[1] Verheiratet, zusammenlebend': 'Relationship',
    '[2] Verheiratet, getrenntlebend': 'Single',
    '[3] Ledig': 'Single',
    '[4] Geschieden, eing. gleichg. Partn. aufgehoben': 'Single',
    '[5] Verwitwet, Lebenspartner/in verstorben': 'Single',
    '[6] Eing. gleichg. Partn., zusammenlebend': 'Relationship',
    '[7] Eing. gleichg. Partn., getrenntlebend': 'Single',
}
df.MARITAL_STATUS.replace(marital_status_dict, inplace=True)
df.MARITAL_STATUS = df.MARITAL_STATUS.astype('category')

## ``REASON_JOB_TERMINATED``

This requires further knowledge of monthly employment. This data is present in ``pkal``.

In [15]:
# As Preuss, Hennecke (2017), we only consider plant closure and
# displacement by employer to be sufficiently exogenous. Other reasons are
# discarded. Rename useful categoires
reason_job_terminated_dict = {
    '[1] Betriebsstillegung, Aufloesung Dienstst.': 'Plant closure',
    '[3] Kuendigung Arbeitgeber': 'Displacement by employer'}
df.REASON_JOB_TERMINATED.cat.rename_categories(reason_job_terminated_dict,
                                               inplace=True)
# Delete useless categories
reason_job_terminated_list = [
    '[2] Eigene Kuendigung', '[4] Aufloesungsvertrag',
    '[5] Ende Befristung', '[6] Verrentung, Pensionierung',
    '[7] Beurlaubung', '[8] Geschaeftsaufgabe (Selbstaendige)',
    '[9] Vorruhestand', '[10] Ende der Ausbildung',
    '[11] Versetzung auf eigenen Wunsch', '[12] Versetzung durch Betrieb',
    '[13] Sonstige Gruende']
df.REASON_JOB_TERMINATED.cat.remove_categories(reason_job_terminated_list,
                                               inplace=True)

# Clean event variables

Before looking at the event variables, it is best to separate the sample in two periods, 2005-2010 and 2010-2015. Here, I outline the following steps.

1. Separate the sample in 2005-2010 and 2010-2015
2. Based on ``INT_MONTH``, delete all events which are not occurring in the period and compute related variables
3. Split data for every event and calculate concurring events based on this month

## Create common representation

In [17]:
for var in EVENT_VARIABLES:
    # Shift var_MONTH_PY in the previous year
    df[var + '_MONTH_PY_SHIFTED'] = df.groupby(
        'ID')[var + '_MONTH_PY'].shift(-1)
    # Use var_MONTH_SY where both agree
    df.loc[df[var + '_MONTH_SY'] == df[var + '_MONTH_PY_SHIFTED'],
           var + '_MONTH'] = df[var + '_MONTH_SY']
    # Use var_MONTH_PY_SHIFTED where var_MONTH_SY is NaN
    df.loc[df[var + '_MONTH_SY'].isnull() & 
           df[var + '_MONTH_PY_SHIFTED'].notnull(),
           var + '_MONTH'] = df[var + '_MONTH_PY_SHIFTED']
    # Use var_MONTH_SY where var_MONTH_PY_SHIFTED is NaN or they
    # disagree because the PY could be more error prone to memory loss
    df.loc[df[var + '_MONTH_SY'].notnull() &
           df[var + '_MONTH_PY_SHIFTED'].isnull(),
           var + '_MONTH'] = df[var + '_MONTH_SY']
    df.loc[df[var + '_MONTH_SY'] != df[var + '_MONTH_PY_SHIFTED'],
           var + '_MONTH'] = df[var + '_MONTH_SY']
    # Make var_MONTH a categorical
    df[var + '_MONTH'] = df[var + '_MONTH'].astype('category')
    df[var + '_MONTH'].cat.set_categories(MONTH_DICT.values(),
                                          ordered=True, inplace=True)

    # Create variable whether var was before the interview to determine
    # timing. Note that, cases where the months of the event and interview
    # coincide are flagged as False.
    df.loc[df[var + '_MONTH'].notnull(),
           var + '_BEFORE_INTERVIEW'] = df[var + '_MONTH'] < df.INT_MONTH
    # There are some cases in which interview and var coincide. An event
    # happened before the interview if var_MONTH_SY is not NaN. The
    # opposite case if var_MONTH_SY is NaN is already flagged as false due
    # to the previous step
    df.loc[(df[var + '_MONTH'] == df.INT_MONTH) &
           df[var + '_MONTH_SY'].notnull(),
           var + '_BEFORE_INTERVIEW'] = True

In [18]:
# Separate the sample in the two periods, 2005-2010 and 2010-2015.
df_2005_2010 = df.loc[df.YEAR.between(2005, 2010)].copy()
df_2010_2015 = df.loc[df.YEAR.between(2010, 2015)].copy()

# Delete events which are not occurring in the specific periods
for var in EVENT_VARIABLES:
    # Delete events before interview in 2005
    df_2005_2010.loc[(df_2005_2010.YEAR == 2005) &
                     df_2005_2010[var + '_BEFORE_INTERVIEW'],
                     var + '_MONTH'] = np.nan
    # Delete events in the first period after the interview
    df_2005_2010.loc[(df_2005_2010.YEAR == 2010) &
                     (df_2005_2010[var + '_BEFORE_INTERVIEW'] == 0),
                     var + '_MONTH'] = np.nan
    # Delete events in the second period before the interview
    df_2010_2015.loc[(df_2010_2015.YEAR == 2010) &
                     df_2010_2015[var + '_BEFORE_INTERVIEW'],
                     var + '_MONTH'] = np.nan
    # Delete events after interview in 2015
    df_2010_2015.loc[(df_2010_2015.YEAR == 2015) &
                     (df_2010_2015[var + '_BEFORE_INTERVIEW'] == 0),
                     var + '_MONTH'] = np.nan

In [19]:
# Create event identifiers, ongoing counts of current events and ongoing
# counts of previous events.
for df_loop in [df_2005_2010, df_2010_2015]:
    for var in EVENT_VARIABLES:
        # Create event identifier
        df_loop.loc[df_loop[var + '_MONTH'].notnull(), 'EVENT_' + var] = True
        # Create ongoing count of events per period
        # astype(float) to convert True to 1.0, cumsum to count, ffill to
        # to overwrite NaNs in following columns with previous values,
        # fillna(0) to convert NaNs at the beginning to zeros.
        df_loop['EVENT_' + var + '_COUNT'] = df_loop.groupby(
            'ID')['EVENT_' + var].transform(
                lambda x: x.astype(float).cumsum().fillna(
                    method='ffill').fillna(0))
        # Create ongoing count of previous events per period
        df_loop['EVENT_' + var + '_COUNT_PREVIOUS'] = (
            df_loop['EVENT_' + var + '_COUNT'] - 1).clip(lower=0)

In [22]:
# Show all observations with the two events in the same month. Luckily,
# there are only 5 :)
count = 0
for df_loop in [df_2005_2010, df_2010_2015]:
    left_var = list(EVENT_VARIABLES)
    for var in EVENT_VARIABLES:
        left_var.remove(var)
        for sub_var in left_var:
            if df_loop[(df_loop[var + '_MONTH'] == df_loop[sub_var + '_MONTH'])].shape[0]:
#                 print(df_loop.loc[df_loop[var + '_MONTH'] == df_loop[sub_var + '_MONTH'],
#                              ['ID', 'YEAR', var + '_MONTH', sub_var + '_MONTH']])
                count += df_loop[(df_loop[var + '_MONTH'] == df_loop[sub_var + '_MONTH'])].shape[0]
count

48

In [23]:
# Count all observations with the two events in the same year.
count = 0
for df_loop in [df_2005_2010, df_2010_2015]:
    left_var = list(EVENT_VARIABLES)
    for var in EVENT_VARIABLES:
        left_var.remove(var)
        for sub_var in left_var:
            if df_loop[df_loop[var + '_MONTH'].notnull() & df_loop[sub_var + '_MONTH'].notnull()].shape[0]:
#                 print(df_loop.loc[df_loop[var + '_MONTH'] == df_loop[sub_var + '_MONTH'],
#                              ['ID', 'YEAR', var + '_MONTH', sub_var + '_MONTH']])
                count += df_loop[df_loop[var + '_MONTH'].notnull() & df_loop[sub_var + '_MONTH'].notnull()].shape[0]
count

457

In [25]:
df.loc[df.LAST_JOB_ENDED_MONTH.notnull(), ['ID', 'YEAR', 'INT_MONTH', 'LAST_JOB_ENDED_MONTH', 'EMPLOYMENT_STATUS']]

,ID,YEAR,INT_MONTH,LAST_JOB_ENDED_MONTH,EMPLOYMENT_STATUS
145,602,2008,April,September,Other
187,901,2014,April,July,Employed
515,2301,2012,February,July,Employed
545,2302,2010,February,November,Employed
1504,6002,2006,July,April,Not Employed
1507,6002,2009,June,February,Not Employed
1509,6002,2011,June,February,Not Employed
2061,8603,2006,August,March,Employed
2064,8603,2009,February,November,Employed
2100,8605,2005,February,October,Employed


In [ ]:
df.DEATH_CHILD_BEFORE_INTERVIEW.value_counts()

In [ ]:
inspect(df_2010_2015, 'DEATH_CHILD', row_cond=(df.ID == 2604801))

In [ ]:
inspect(df_2010_2015, 'DEATH_CHILD', row_cond=(df.ID == 2604801))

In [ ]:
inspect(df_2010_2015, 'DEATH_CHILD', row_cond=(df.EVENT_DEATH_CHILD_COUNT >= 2))

In [ ]:
df.loc[df.EVENT_DEATH_CHILD.notnull()]

In [ ]:
asdsad 2

In [ ]:
event_list = ['DEATH_CHILD', 'DEATH_FATHER', 'DEATH_HH_PERSON',
              'DEATH_MOTHER', 'DEATH_PARTNER', 'DIVORCED',
              'HH_COMP_CHANGE', 'SEPARATED']

for df in [df_2005_2010, df_2010_2015]:
    for var in event_list:
        for sub_var in [i for i in event_list if i not var]:
            # Create variables indicating whether the event happened before or after 

In [ ]:
def inspect(df, event, row_cond=None):
    related_columns = [i for i in df if event in i]
    if row_cond is None:
        return df.loc[:, ['ID', 'YEAR'] + related_columns]
    else:
        return df.loc[row_cond, ['ID', 'YEAR'] + related_columns]